<a href="https://colab.research.google.com/github/MitjaGo/YT/blob/main/You_tube_Downloader_MP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# ==========================
# STEP 1: Install dependencies
# ==========================
!pip install -q yt-dlp pydub mutagen ffmpeg-python requests

# ==========================
# STEP 2: Import libraries
# ==========================
import os
import json
import subprocess
import requests
import shutil
from pydub import AudioSegment, silence
from mutagen.easyid3 import EasyID3
from mutagen.id3 import ID3, APIC
from google.colab import files

# ==========================
# STEP 3: Get YouTube URL from user input
# ==========================
youtube_url = input("📥 Paste the YouTube URL here and press ENTER: ").strip()

# ==========================
# STEP 4: Download audio with yt-dlp
# ==========================
os.makedirs("downloads", exist_ok=True)
output_path = "downloads/youtube_audio.%(ext)s"

print("⬇️ Downloading audio...")
!yt-dlp -x --audio-format mp3 -o "{output_path}" "{youtube_url}" -q

downloaded_files = os.listdir("downloads")
audio_file = [f for f in downloaded_files if f.endswith(".mp3")][0]
audio_path = os.path.join("downloads", audio_file)
print("✅ Audio downloaded:", audio_path)

# Load audio
audio = AudioSegment.from_file(audio_path)

# ==========================
# STEP 5: Download YouTube thumbnail & metadata
# ==========================
print("⬇️ Fetching video info and thumbnail...")
result = subprocess.run(['yt-dlp', '-J', youtube_url], capture_output=True, text=True)
info = json.loads(result.stdout)
thumbnail_url = info['thumbnail']
video_title = info['title']
video_author = info.get('uploader', 'Unknown')

thumb_data = requests.get(thumbnail_url).content
thumb_path = "downloads/thumbnail.jpg"
with open(thumb_path, "wb") as f:
    f.write(thumb_data)
print("🖼️ Thumbnail downloaded.")

# ==========================
# STEP 6: Split audio by silence and tag files
# ==========================
os.makedirs("sections", exist_ok=True)

MIN_SILENCE_LEN = 1000  # milliseconds
SILENCE_THRESH = audio.dBFS - 16  # relative to audio loudness

chunks = silence.split_on_silence(
    audio,
    min_silence_len=MIN_SILENCE_LEN,
    silence_thresh=SILENCE_THRESH,
    keep_silence=500
)

if not chunks:
    print("⚠️ No silence detected — exporting full audio as a single section.")
    chunks = [audio]

sections = []
use_custom_titles = False

# Check if multiple sections would have the same title
if len(chunks) > 1:
    sample_titles = [video_title] * len(chunks)
    if len(set(sample_titles)) == 1:
        use_custom_titles = True
        print("⚠️ Multiple sections detected with the same title. You can enter custom titles for each section.")

for i, chunk in enumerate(chunks, 1):
    temp_filename = f"sections/section_{i}.mp3"
    chunk.export(temp_filename, format="mp3")

    # Determine section title
    if use_custom_titles:
        print(f"\n🎵 Section {i} duration: {len(chunk)//1000} seconds")
        custom_title = input(f"Enter a custom title for Section {i} (leave empty for default '{video_title} - Part {i}'): ").strip()
        section_title = custom_title if custom_title else f"{video_title} - Part {i}"
    else:
        section_title = f"{video_title} - Part {i}" if len(chunks) > 1 else video_title

    # Add ID3 metadata + album art
    audiofile = EasyID3(temp_filename)
    audiofile['title'] = section_title
    audiofile['artist'] = video_author
    audiofile['album'] = video_title
    audiofile['tracknumber'] = str(i)
    audiofile.save()

    # Embed thumbnail as ID3 album art
    tags = ID3(temp_filename)
    with open(thumb_path, 'rb') as img:
        tags['APIC'] = APIC(
            encoding=3,        # UTF-8
            mime='image/jpeg', # JPEG format
            type=3,            # Cover(front)
            desc='Cover',
            data=img.read()
        )
    tags.save()

    # Rename file to "Title - Artist.mp3"
    safe_title = audiofile['title'][0].replace("/", "_").replace("\\", "_")
    safe_artist = audiofile['artist'][0].replace("/", "_").replace("\\", "_")
    new_filename = f"sections/{safe_title} - {safe_artist}.mp3"
    os.rename(temp_filename, new_filename)

    sections.append(new_filename)
    print(f"💾 Section {i} exported, tagged, and renamed to: {new_filename}")

# ==========================
# STEP 7: Summary file
# ==========================
with open("sections_info.txt", "w") as f:
    for i, s in enumerate(sections, 1):
        f.write(f"{i}. {s}\n")

print("📝 Summary file created: sections_info.txt")

# ==========================
# STEP 8: Zip all sections for download
# ==========================
shutil.make_archive("sections", "zip", "sections")
print("📦 All sections zipped as sections.zip")

# Download summary and zip
files.download("sections_info.txt")
files.download("sections.zip")


📥 Paste the YouTube URL here and press ENTER: https://www.youtube.com/watch?v=_igNxWonKfs
⬇️ Downloading audio...
         player = https://www.youtube.com/s/player/6e4dbefe/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U
✅ Audio downloaded: downloads/youtube_audio.mp3
⬇️ Fetching video info and thumbnail...
🖼️ Thumbnail downloaded.
💾 Section 1 exported, tagged, and renamed to: sections/Zucchero - Amor Che Muovi Il Sole - Zucchero.mp3
📝 Summary file created: sections_info.txt
📦 All sections zipped as sections.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>